In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import time
import copy

# --- 1. 参数配置 ---
DATA_DIR = 'dataset'  # 刚才生成的文件夹路径
NUM_CLASSES = 6       # 结果是6分类
BATCH_SIZE = 16       #集显所以设置小一点 
NUM_EPOCHS = 10       #同上
LEARNING_RATE = 0.001
DEVICE = torch.device("cpu")

def main():
    print(f"使用设备: {DEVICE}")

    # --- 2. 数据增强与预处理 ---
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),      # ResNet 标准输入
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 均值方差
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # --- 3. 加载数据集 ---
    image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                      for x in ['train', 'val']}
    
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
                   for x in ['train', 'val']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    
    print(f"类别列表: {class_names}")
    print(f"训练集数量: {dataset_sizes['train']}, 验证集数量: {dataset_sizes['val']}")

    # --- 4. 搭建模型 (迁移学习) ---
    # 下载预训练的 ResNet18
    model = models.resnet18(pretrained=True)
    
    # 获取最后一个全连接层的输入特征数
    num_ftrs = model.fc.in_features
    
    # 修改全连接层，输出节点改为 6 
    model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
    
    model = model.to(DEVICE)

    # --- 5. 定义损失函数与优化器 ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # 学习率衰减策略：7个epoch学习率乘以0.1 
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # --- 6. 训练循环 ---
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    start_time = time.time()

    for epoch in range(NUM_EPOCHS):
        print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式 (启用 Dropout, BN 更新)
            else:
                model.eval()   # 验证模式 (冻结 BN)

            running_loss = 0.0
            running_corrects = 0

            # 批次迭代
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                # 梯度清零
                optimizer.zero_grad()

                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 反向传播与优化 (仅在训练阶段)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 统计 loss 和 准确率
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 深度拷贝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - start_time
    print(f'\n训练完成! 耗时: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'最佳验证集准确率: {best_acc:.4f}')

    # --- 7. 保存最佳模型 ---
    model.load_state_dict(best_model_wts)
    save_path = 'neu_det_resnet18.pth'
    torch.save(model.state_dict(), save_path)
    print(f"模型已保存至: {save_path}")

if __name__ == '__main__':
    main()

使用设备: cpu
类别列表: ['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']
训练集数量: 1440, 验证集数量: 360


D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\yunheishere/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|█████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:05<00:00, 9.13MB/s]



Epoch 1/10
----------
train Loss: 0.5697 Acc: 0.8299
val Loss: 2.0889 Acc: 0.6944

Epoch 2/10
----------
train Loss: 0.2998 Acc: 0.9062
val Loss: 0.1893 Acc: 0.9444

Epoch 3/10
----------
train Loss: 0.2356 Acc: 0.9271
val Loss: 4.1680 Acc: 0.7417

Epoch 4/10
----------
train Loss: 0.1901 Acc: 0.9451
val Loss: 0.1003 Acc: 0.9694

Epoch 5/10
----------
train Loss: 0.2060 Acc: 0.9444
val Loss: 0.0731 Acc: 0.9778

Epoch 6/10
----------
train Loss: 0.0581 Acc: 0.9861
val Loss: 0.0343 Acc: 0.9889

Epoch 7/10
----------
train Loss: 0.0582 Acc: 0.9799
val Loss: 0.0369 Acc: 0.9833

Epoch 8/10
----------
train Loss: 0.0223 Acc: 0.9958
val Loss: 0.0165 Acc: 0.9944

Epoch 9/10
----------
train Loss: 0.0171 Acc: 0.9951
val Loss: 0.0146 Acc: 0.9944

Epoch 10/10
----------
train Loss: 0.0124 Acc: 0.9986
val Loss: 0.0132 Acc: 0.9944

训练完成! 耗时: 26m 9s
最佳验证集准确率: 0.9944
模型已保存至: neu_det_resnet18.pth
